# Chapter 3: Advanced SQL

This chapter covers advanced SQL concepts for complex data analysis, optimization, and sophisticated querying techniques.

## Topics Covered:
1. Subqueries and Correlated Queries
2. Common Table Expressions (CTEs)
3. Window Functions in Detail
4. Advanced Data Manipulation
5. Query Optimization Techniques
6. Real-world Analysis Scenarios

## Prerequisites

Ensure you've completed Chapters 1 & 2 and have the database connection established.

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import numpy as np

# Connect to the database
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# Add some additional sample data for advanced examples
cursor.execute('''
CREATE TABLE IF NOT EXISTS employee_performance (
    emp_id INTEGER,
    year INTEGER,
    performance_score DECIMAL(3,2),
    bonus DECIMAL(10,2),
    PRIMARY KEY (emp_id, year),
    FOREIGN KEY (emp_id) REFERENCES employees (emp_id)
)
''')

# Insert performance data
performance_data = [
    (1, 2022, 4.2, 5000), (1, 2023, 4.5, 6000),
    (2, 2022, 4.8, 8000), (2, 2023, 4.7, 7500),
    (3, 2022, 3.9, 3500), (3, 2023, 4.1, 4000),
    (4, 2022, 4.6, 7000), (4, 2023, 4.8, 8500),
    (5, 2022, 4.4, 6500), (5, 2023, 4.3, 6000),
    (6, 2022, 4.0, 4500), (6, 2023, 4.2, 5000),
    (7, 2022, 4.7, 7500), (7, 2023, 4.9, 9000),
    (8, 2022, 4.1, 4000), (8, 2023, 4.3, 4500)
]

cursor.executemany('INSERT OR REPLACE INTO employee_performance VALUES (?, ?, ?, ?)', performance_data)
conn.commit()

print("Advanced examples database setup complete!")

## 1. Subqueries and Correlated Queries

Subqueries allow you to use the result of one query inside another query.

In [ ]:
# Example 1: Simple subquery - Employees earning above average
print("Employees earning above company average:")
df = pd.read_sql_query("""
SELECT 
    e.first_name,
    e.last_name,
    e.salary,
    d.dept_name,
    (SELECT AVG(salary) FROM employees) as company_avg
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
WHERE e.salary > (SELECT AVG(salary) FROM employees)
ORDER BY e.salary DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Correlated subquery - Employees earning above their department average
print("Employees earning above their department average:")
df = pd.read_sql_query("""
SELECT 
    e.first_name,
    e.last_name,
    e.salary,
    d.dept_name,
    (SELECT AVG(e2.salary) 
     FROM employees e2 
     WHERE e2.dept_id = e.dept_id) as dept_avg
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
WHERE e.salary > (
    SELECT AVG(e2.salary) 
    FROM employees e2 
    WHERE e2.dept_id = e.dept_id
)
ORDER BY d.dept_name, e.salary DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 3: EXISTS clause - Departments with high-budget projects
print("Departments that have projects with budget > 200,000:")
df = pd.read_sql_query("""
SELECT d.dept_name, d.location
FROM departments d
WHERE EXISTS (
    SELECT 1 
    FROM projects p 
    WHERE p.dept_id = d.dept_id 
    AND p.budget > 200000
)
""", conn)
display(df)

## 2. Common Table Expressions (CTEs)

CTEs provide a way to write more readable and maintainable complex queries.

In [ ]:
# Example 1: Basic CTE - Department statistics
print("Department analysis using CTE:")
df = pd.read_sql_query("""
WITH dept_stats AS (
    SELECT 
        d.dept_id,
        d.dept_name,
        COUNT(e.emp_id) as employee_count,
        AVG(e.salary) as avg_salary,
        SUM(e.salary) as total_salary
    FROM departments d
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    GROUP BY d.dept_id, d.dept_name
)
SELECT 
    dept_name,
    employee_count,
    ROUND(avg_salary, 2) as avg_salary,
    total_salary,
    ROUND(total_salary / (SELECT SUM(total_salary) FROM dept_stats) * 100, 2) as salary_percentage
FROM dept_stats
ORDER BY total_salary DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Recursive CTE simulation - Employee hierarchy
print("Performance trends using CTE:")
df = pd.read_sql_query("""
WITH performance_trends AS (
    SELECT 
        e.emp_id,
        e.first_name || ' ' || e.last_name as full_name,
        d.dept_name,
        AVG(p.performance_score) as avg_performance,
        SUM(p.bonus) as total_bonus,
        COUNT(p.year) as years_reviewed
    FROM employees e
    INNER JOIN departments d ON e.dept_id = d.dept_id
    INNER JOIN employee_performance p ON e.emp_id = p.emp_id
    GROUP BY e.emp_id, e.first_name, e.last_name, d.dept_name
),
company_stats AS (
    SELECT 
        AVG(avg_performance) as company_avg_performance,
        AVG(total_bonus) as company_avg_bonus
    FROM performance_trends
)
SELECT 
    pt.full_name,
    pt.dept_name,
    ROUND(pt.avg_performance, 2) as avg_performance,
    pt.total_bonus,
    ROUND(cs.company_avg_performance, 2) as company_avg_performance,
    CASE 
        WHEN pt.avg_performance > cs.company_avg_performance THEN 'Above Average'
        ELSE 'Below Average'
    END as performance_rating
FROM performance_trends pt
CROSS JOIN company_stats cs
ORDER BY pt.avg_performance DESC
""", conn)
display(df)

## 3. Advanced Window Functions

Window functions perform calculations across a set of rows related to the current row.

In [ ]:
# Example 1: Ranking functions
print("Employee rankings across multiple dimensions:")
df = pd.read_sql_query("""
SELECT 
    e.first_name || ' ' || e.last_name as full_name,
    d.dept_name,
    e.salary,
    ROW_NUMBER() OVER (ORDER BY e.salary DESC) as overall_rank,
    RANK() OVER (PARTITION BY d.dept_name ORDER BY e.salary DESC) as dept_rank,
    DENSE_RANK() OVER (ORDER BY e.salary DESC) as dense_rank,
    NTILE(3) OVER (ORDER BY e.salary DESC) as salary_tier
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
ORDER BY e.salary DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Analytical functions with performance data
print("Performance analysis with trends:")
df = pd.read_sql_query("""
SELECT 
    e.first_name || ' ' || e.last_name as full_name,
    d.dept_name,
    p.year,
    p.performance_score,
    LAG(p.performance_score) OVER (PARTITION BY e.emp_id ORDER BY p.year) as prev_score,
    p.performance_score - LAG(p.performance_score) OVER (PARTITION BY e.emp_id ORDER BY p.year) as score_change,
    AVG(p.performance_score) OVER (PARTITION BY d.dept_name) as dept_avg_performance,
    MAX(p.performance_score) OVER (PARTITION BY d.dept_name) as dept_max_performance
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
INNER JOIN employee_performance p ON e.emp_id = p.emp_id
ORDER BY d.dept_name, e.last_name, p.year
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 3: Running totals and moving averages
print("Financial analysis with running totals:")
df = pd.read_sql_query("""
SELECT 
    d.dept_name,
    p.project_name,
    p.budget,
    SUM(p.budget) OVER (PARTITION BY d.dept_name ORDER BY p.budget) as running_total,
    AVG(p.budget) OVER (PARTITION BY d.dept_name) as dept_avg_budget,
    p.budget - AVG(p.budget) OVER (PARTITION BY d.dept_name) as budget_vs_avg,
    PERCENT_RANK() OVER (ORDER BY p.budget) as budget_percentile
FROM projects p
INNER JOIN departments d ON p.dept_id = d.dept_id
ORDER BY d.dept_name, p.budget
""", conn)
display(df)

## 4. Advanced Data Manipulation

Explore sophisticated data transformation and analysis techniques.

In [ ]:
# Example 1: Pivot-like analysis using CASE statements
print("Employee distribution by salary ranges:")
df = pd.read_sql_query("""
SELECT 
    d.dept_name,
    COUNT(*) as total_employees,
    SUM(CASE WHEN e.salary < 70000 THEN 1 ELSE 0 END) as below_70k,
    SUM(CASE WHEN e.salary BETWEEN 70000 AND 85000 THEN 1 ELSE 0 END) as mid_range,
    SUM(CASE WHEN e.salary > 85000 THEN 1 ELSE 0 END) as above_85k,
    ROUND(
        AVG(CASE WHEN e.salary > 85000 THEN e.salary END), 2
    ) as avg_high_earner_salary
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
ORDER BY total_employees DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Complex conditional aggregation
print("Department performance matrix:")
df = pd.read_sql_query("""
WITH dept_metrics AS (
    SELECT 
        d.dept_name,
        COUNT(DISTINCT e.emp_id) as employee_count,
        COUNT(DISTINCT pr.project_id) as project_count,
        AVG(e.salary) as avg_salary,
        COALESCE(SUM(pr.budget), 0) as total_budget,
        AVG(perf.performance_score) as avg_performance
    FROM departments d
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    LEFT JOIN projects pr ON d.dept_id = pr.dept_id
    LEFT JOIN employee_performance perf ON e.emp_id = perf.emp_id
    GROUP BY d.dept_name
)
SELECT 
    dept_name,
    employee_count,
    project_count,
    ROUND(avg_salary, 2) as avg_salary,
    total_budget,
    ROUND(avg_performance, 2) as avg_performance,
    CASE 
        WHEN avg_performance > 4.5 AND avg_salary > 80000 THEN 'High Performance, High Cost'
        WHEN avg_performance > 4.5 THEN 'High Performance, Standard Cost'
        WHEN avg_salary > 80000 THEN 'High Cost, Standard Performance'
        ELSE 'Standard'
    END as department_profile,
    ROUND(total_budget / NULLIF(employee_count, 0), 2) as budget_per_employee
FROM dept_metrics
ORDER BY avg_performance DESC, avg_salary DESC
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 3: Time-based analysis
print("Year-over-year performance comparison:")
df = pd.read_sql_query("""
WITH yearly_performance AS (
    SELECT 
        e.emp_id,
        e.first_name || ' ' || e.last_name as full_name,
        d.dept_name,
        p.year,
        p.performance_score,
        p.bonus,
        LAG(p.performance_score) OVER (PARTITION BY e.emp_id ORDER BY p.year) as prev_year_score,
        LAG(p.bonus) OVER (PARTITION BY e.emp_id ORDER BY p.year) as prev_year_bonus
    FROM employees e
    INNER JOIN departments d ON e.dept_id = d.dept_id
    INNER JOIN employee_performance p ON e.emp_id = p.emp_id
)
SELECT 
    full_name,
    dept_name,
    year,
    performance_score as current_score,
    prev_year_score,
    ROUND(performance_score - COALESCE(prev_year_score, 0), 2) as score_improvement,
    bonus as current_bonus,
    prev_year_bonus,
    bonus - COALESCE(prev_year_bonus, 0) as bonus_change,
    CASE 
        WHEN performance_score > COALESCE(prev_year_score, 0) THEN 'Improved'
        WHEN performance_score = prev_year_score THEN 'Stable'
        ELSE 'Declined'
    END as performance_trend
FROM yearly_performance
WHERE year = 2023  -- Focus on 2023 with comparison to 2022
ORDER BY score_improvement DESC
""", conn)
display(df)

## 5. Query Optimization Techniques

Learn techniques to write efficient SQL queries for better performance.

In [ ]:
# Example 1: Using indexes (simulation with EXPLAIN QUERY PLAN)
print("Query plan analysis:")
df = pd.read_sql_query("""
EXPLAIN QUERY PLAN
SELECT e.first_name, e.last_name, d.dept_name
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
WHERE e.salary > 80000
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Efficient vs inefficient query patterns
print("Optimized query - using EXISTS instead of IN:")

# More efficient approach
df1 = pd.read_sql_query("""
SELECT d.dept_name
FROM departments d
WHERE EXISTS (
    SELECT 1 FROM employees e 
    WHERE e.dept_id = d.dept_id AND e.salary > 85000
)
""", conn)
print("Departments with high earners (using EXISTS):")
display(df1)

# Alternative approach (for comparison)
df2 = pd.read_sql_query("""
SELECT DISTINCT d.dept_name
FROM departments d
INNER JOIN employees e ON d.dept_id = e.dept_id
WHERE e.salary > 85000
""", conn)
print("\nSame result using INNER JOIN:")
display(df2)

print("\n" + "="*50 + "\n")

# Example 3: Using LIMIT for large datasets
print("Top performers by department (limited results):")
df = pd.read_sql_query("""
WITH ranked_employees AS (
    SELECT 
        e.first_name || ' ' || e.last_name as full_name,
        d.dept_name,
        e.salary,
        RANK() OVER (PARTITION BY d.dept_name ORDER BY e.salary DESC) as rank
    FROM employees e
    INNER JOIN departments d ON e.dept_id = d.dept_id
)
SELECT full_name, dept_name, salary
FROM ranked_employees
WHERE rank <= 2  -- Top 2 per department
ORDER BY dept_name, rank
""", conn)
display(df)

## 6. Real-world Analysis Scenarios

Apply advanced SQL techniques to solve complex business questions.

In [ ]:
# Scenario 1: Executive Dashboard Query
print("Executive Dashboard - Company Overview:")
df = pd.read_sql_query("""
WITH company_metrics AS (
    SELECT 
        COUNT(DISTINCT d.dept_id) as total_departments,
        COUNT(DISTINCT e.emp_id) as total_employees,
        COUNT(DISTINCT p.project_id) as total_projects,
        SUM(e.salary) as total_payroll,
        SUM(p.budget) as total_project_budget,
        AVG(perf.performance_score) as avg_company_performance
    FROM departments d
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    LEFT JOIN projects p ON d.dept_id = p.dept_id
    LEFT JOIN employee_performance perf ON e.emp_id = perf.emp_id
),
top_performers AS (
    SELECT COUNT(*) as high_performers
    FROM employees e
    INNER JOIN employee_performance perf ON e.emp_id = perf.emp_id
    WHERE perf.performance_score > 4.5
),
dept_efficiency AS (
    SELECT 
        AVG(total_budget / NULLIF(employee_count, 0)) as avg_budget_per_employee
    FROM (
        SELECT 
            d.dept_id,
            COUNT(e.emp_id) as employee_count,
            COALESCE(SUM(p.budget), 0) as total_budget
        FROM departments d
        LEFT JOIN employees e ON d.dept_id = e.dept_id
        LEFT JOIN projects p ON d.dept_id = p.dept_id
        GROUP BY d.dept_id
    )
)
SELECT 
    cm.total_departments,
    cm.total_employees,
    cm.total_projects,
    cm.total_payroll,
    cm.total_project_budget,
    cm.total_payroll + cm.total_project_budget as total_investment,
    ROUND(cm.avg_company_performance, 2) as avg_performance,
    tp.high_performers,
    ROUND(tp.high_performers * 100.0 / cm.total_employees, 1) as high_performer_percentage,
    ROUND(de.avg_budget_per_employee, 2) as avg_budget_per_employee
FROM company_metrics cm
CROSS JOIN top_performers tp
CROSS JOIN dept_efficiency de
""", conn)
display(df)

print("\n" + "="*50 + "\n")

# Scenario 2: Talent Management Analysis
print("Talent Management - Risk Assessment:")
df = pd.read_sql_query("""
WITH employee_analysis AS (
    SELECT 
        e.emp_id,
        e.first_name || ' ' || e.last_name as full_name,
        d.dept_name,
        e.salary,
        e.hire_date,
        AVG(perf.performance_score) as avg_performance,
        SUM(perf.bonus) as total_bonus,
        CASE 
            WHEN DATE('now') > DATE(e.hire_date, '+5 years') THEN 'Senior'
            WHEN DATE('now') > DATE(e.hire_date, '+2 years') THEN 'Mid-level'
            ELSE 'Junior'
        END as tenure_level,
        CASE 
            WHEN e.salary > (SELECT AVG(salary) * 1.2 FROM employees) THEN 'High'
            WHEN e.salary > (SELECT AVG(salary) * 0.8 FROM employees) THEN 'Medium'
            ELSE 'Low'
        END as compensation_level
    FROM employees e
    INNER JOIN departments d ON e.dept_id = d.dept_id
    LEFT JOIN employee_performance perf ON e.emp_id = perf.emp_id
    GROUP BY e.emp_id, e.first_name, e.last_name, d.dept_name, e.salary, e.hire_date
)
SELECT 
    full_name,
    dept_name,
    tenure_level,
    compensation_level,
    ROUND(avg_performance, 2) as avg_performance,
    total_bonus,
    CASE 
        WHEN avg_performance > 4.5 AND compensation_level = 'Low' THEN 'Flight Risk - High Performer, Low Pay'
        WHEN avg_performance < 3.5 AND compensation_level = 'High' THEN 'Performance Concern - Low Performance, High Pay'
        WHEN avg_performance > 4.5 AND tenure_level = 'Senior' THEN 'Key Talent - Retain'
        WHEN avg_performance < 3.5 THEN 'Development Needed'
        ELSE 'Standard'
    END as talent_status,
    salary
FROM employee_analysis
ORDER BY 
    CASE 
        WHEN avg_performance > 4.5 AND compensation_level = 'Low' THEN 1
        WHEN avg_performance < 3.5 AND compensation_level = 'High' THEN 2
        WHEN avg_performance > 4.5 AND tenure_level = 'Senior' THEN 3
        ELSE 4
    END,
    avg_performance DESC
""", conn)
display(df)

## Practice Exercises - Chapter 3

Challenge yourself with these advanced SQL problems:

### Exercise Questions:
1. **Create a CTE that identifies the most improved employee in each department (2022 vs 2023)**
2. **Write a query using window functions to find employees whose salary is in the top 25% company-wide**
3. **Build a comprehensive department scorecard using subqueries and aggregations**
4. **Use EXISTS to find departments that have both high-budget projects (>150k) and high-performing employees (>4.5 avg)**
5. **Create a query that shows month-over-month trends (simulate monthly data)**

In [ ]:
# Advanced Practice Area

# Exercise 1: Most improved employee per department
print("Exercise 1: Most improved employees by department (2022 vs 2023)")
query1 = """
WITH performance_improvement AS (
    SELECT 
        e.emp_id,
        e.first_name || ' ' || e.last_name as full_name,
        d.dept_name,
        p2023.performance_score as score_2023,
        p2022.performance_score as score_2022,
        p2023.performance_score - p2022.performance_score as improvement,
        RANK() OVER (PARTITION BY d.dept_name ORDER BY (p2023.performance_score - p2022.performance_score) DESC) as improvement_rank
    FROM employees e
    INNER JOIN departments d ON e.dept_id = d.dept_id
    INNER JOIN employee_performance p2023 ON e.emp_id = p2023.emp_id AND p2023.year = 2023
    INNER JOIN employee_performance p2022 ON e.emp_id = p2022.emp_id AND p2022.year = 2022
)
SELECT 
    dept_name,
    full_name,
    score_2022,
    score_2023,
    ROUND(improvement, 2) as improvement
FROM performance_improvement
WHERE improvement_rank = 1
ORDER BY improvement DESC
"""
df = pd.read_sql_query(query1, conn)
display(df)

print("\n" + "="*50 + "\n")

# Exercise 2: Top 25% earners using window functions
print("Exercise 2: Employees in top 25% salary range")
query2 = """
SELECT 
    e.first_name || ' ' || e.last_name as full_name,
    d.dept_name,
    e.salary,
    NTILE(4) OVER (ORDER BY e.salary DESC) as salary_quartile,
    PERCENT_RANK() OVER (ORDER BY e.salary DESC) as salary_percentile,
    ROUND((SELECT AVG(salary) FROM employees), 2) as company_avg
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
WHERE NTILE(4) OVER (ORDER BY e.salary DESC) = 1
ORDER BY e.salary DESC
"""
df = pd.read_sql_query(query2, conn)
display(df)

print("\n" + "="*50 + "\n")

# Exercise 3: Comprehensive department scorecard
print("Exercise 3: Department Performance Scorecard")
query3 = """
WITH dept_scorecard AS (
    SELECT 
        d.dept_name,
        COUNT(DISTINCT e.emp_id) as employee_count,
        ROUND(AVG(e.salary), 2) as avg_salary,
        COUNT(DISTINCT p.project_id) as project_count,
        COALESCE(SUM(p.budget), 0) as total_budget,
        ROUND(AVG(perf.performance_score), 2) as avg_performance,
        SUM(perf.bonus) as total_bonuses,
        ROUND(COALESCE(SUM(p.budget), 0) / NULLIF(COUNT(DISTINCT e.emp_id), 0), 2) as budget_per_employee
    FROM departments d
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    LEFT JOIN projects p ON d.dept_id = p.dept_id
    LEFT JOIN employee_performance perf ON e.emp_id = perf.emp_id
    GROUP BY d.dept_name
),
company_benchmarks AS (
    SELECT 
        AVG(avg_salary) as company_avg_salary,
        AVG(avg_performance) as company_avg_performance,
        AVG(budget_per_employee) as company_avg_budget_per_emp
    FROM dept_scorecard
)
SELECT 
    ds.dept_name,
    ds.employee_count,
    ds.avg_salary,
    ds.project_count,
    ds.total_budget,
    ds.avg_performance,
    ds.budget_per_employee,
    CASE 
        WHEN ds.avg_performance > cb.company_avg_performance * 1.1 THEN 'Excellent'
        WHEN ds.avg_performance > cb.company_avg_performance * 0.9 THEN 'Good'
        ELSE 'Needs Improvement'
    END as performance_rating,
    CASE 
        WHEN ds.budget_per_employee > cb.company_avg_budget_per_emp * 1.2 THEN 'High Investment'
        WHEN ds.budget_per_employee > cb.company_avg_budget_per_emp * 0.8 THEN 'Standard Investment'
        ELSE 'Low Investment'
    END as investment_level
FROM dept_scorecard ds
CROSS JOIN company_benchmarks cb
ORDER BY ds.avg_performance DESC, ds.total_budget DESC
"""
df = pd.read_sql_query(query3, conn)
display(df)

# Continue with remaining exercises...

In [ ]:
# Clean up connections
conn.close()
print("Database connections closed. Advanced SQL chapter complete!")

## Chapter Summary

Congratulations! You've mastered advanced SQL concepts:

✅ **Subqueries & Correlated Queries** - Complex nested query patterns  
✅ **Common Table Expressions (CTEs)** - Readable and maintainable complex queries  
✅ **Advanced Window Functions** - Analytical functions for sophisticated analysis  
✅ **Data Manipulation** - Complex transformations and conditional logic  
✅ **Query Optimization** - Performance considerations and best practices  
✅ **Real-world Scenarios** - Business intelligence and analytics applications  

### Key Advanced Concepts Mastered:
- **Correlated subqueries** for row-by-row comparisons
- **CTEs** for breaking down complex logic
- **Window functions** for rankings and running calculations
- **Advanced aggregations** with conditional logic
- **Performance optimization** techniques
- **Business intelligence** query patterns

### Professional Development:
You now have the SQL skills to:
- Build executive dashboards and reports
- Perform complex data analysis
- Optimize query performance
- Handle real-world business scenarios
- Work with large datasets efficiently

### Next Steps for Continued Learning:
1. **Database-Specific Features** - Learn PostgreSQL, MySQL, or SQL Server specific functions
2. **Data Warehousing** - Explore OLAP, star schemas, and dimensional modeling
3. **Big Data SQL** - Learn Spark SQL, Presto, or other distributed SQL engines
4. **Database Administration** - Indexing, query tuning, and performance monitoring
5. **Integration Projects** - Combine SQL with Python, R, or BI tools

**🎉 You've completed the SQL Learning Journey! You're now equipped with professional-level SQL skills.**